In [ ]:
import pandas as pd
import os

In [3]:
folder_path = 'data/'

dataframes = []  # list to hold all dataframes

for file in os.listdir(folder_path):
    if file.endswith('.csv'):
        year = file.split('_')[-1].split('.')[0]
        df = pd.read_csv(os.path.join(folder_path, file))

        df['Year'] = year

        dataframes.append(df)

combined_df = pd.concat(dataframes, ignore_index=True)

print("Combined Shape: ", combined_df.shape)
combined_df.head()

Combined Shape:  (114055, 6)


,COMMODITY,COUNTRY,UNIT,QUANTITY,VALUE(US$ million),Year
0,TEA,AFGHANISTAN,KGS,996994.0,2.88,201718
1,TEA,ALBANIA,KGS,250065.0,0.82,201718
2,TEA,ALGERIA,KGS,109640.0,0.21,201718
3,TEA,ANDORRA,KGS,32692.0,0.15,201718
4,TEA,ANGOLA,KGS,11361.0,0.07,201718


In [4]:
combined_df.rename(columns={
    'COMMODITY': 'Commodity',
    'COUNTRY': 'Country',
    'UNIT': 'Unit',
    'QUANTITY': 'Quantity',
    'VALUE(US$ million)': 'Value_USD_Million',
}, inplace = True)

In [5]:
combined_df['Value_USD_Million'] = (combined_df['Value_USD_Million']
                                   .replace(',','',regex=True)
                                   .astype(float))

In [6]:
combined_df['Quantity'] = pd.to_numeric(combined_df['Quantity']
                          .replace(',','',regex=True), errors='coerce')

In [6]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114055 entries, 0 to 114054
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   COMMODITY           114055 non-null  object 
 1   COUNTRY             114050 non-null  object 
 2   UNIT                67751 non-null   object 
 3   QUANTITY            67784 non-null   float64
 4   VALUE(US$ million)  114055 non-null  float64
 5   Year                114055 non-null  object 
dtypes: float64(2), object(4)
memory usage: 5.2+ MB


In [9]:
print(f"Shape of combined dataset: {combined_df.shape}")
print("\nColumns and data types:")
print(combined_df.dtypes)

Shape of combined dataset: (114055, 6)

Columns and data types:
Commodity             object
Country               object
Unit                  object
Quantity             float64
Value_USD_Million    float64
Year                  object
dtype: object


In [7]:
print("Missing values per column: ")
print(combined_df.isna().sum())

Missing values per column: 
Commodity                0
Country                  5
Unit                 46304
Quantity             46271
Value_USD_Million        0
Year                     0
dtype: int64


In [ ]:
# As seen above, the features 'UNIT' and 'QUANTITY' are almost empty, which leads to a huge chunk of information not available.
# So, in the below cell, two approaches are discussed, which can be used i.e., (A, B)

In [11]:
# # A: Keep all data that is only preferred for value analysis, remove rest of the data
# # If your main focus is monetary export trends, you can ignore UNIT and QUANTITY safely.

# df_exports = combined_df.drop(columns=['UNIT', 'QUANTITY'])



# B: Separate Physical vs Non-Physical Commodities
# If you want to also analyze quantities (e.g., “exports measured in tonnes”), split dataset:

# Physical commodities (with quantity and unit)
df_with_quantity = combined_df.dropna(subset = ['Quantity','Unit']).copy()

# Non-physical / value-only commodities
df_without_quantity = combined_df[combined_df['Quantity'].isna() | combined_df['Unit'].isna()].copy()

# Now you can:
# Analyze volume-based trends (df_with_quantity).
# Analyze value-only trends (df_without_quantity).